<style>       
    hr{
        height: 4px;
        background-color: rgb(247,148,9);
        border: none;
    }
</style>
<div style="color=white;
           display:fill;
           border-radius:5px;
           background-color:rgb(34,41,49)">
<hr>
<div align="right"><i>BTE5034 - Digital Signal Processing &nbsp;</i></div>
<div align="right">EIT - BFH &nbsp;</div>

<div style="clear: both; font-size: 30pt; font-weight: bold;">
    Discrete-time Filters
</div>
<hr>
</div>

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import IPython
import scipy.signal as sp
from scipy.io import wavfile

# interactivity library:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [12]:
plt.rcParams["figure.figsize"] = (14,4)

# Denoising

In a denoising scenario we have a "clean" signal $x[n]$ that has been corrupted by an additive noise signal $\eta[n]$; we only have access to $\hat{x}[n] = x[n] + \eta[n]$ and we would like to recover $x[n]$.

In general, without further assumptions, this is not a solvable problem. However, it is generally the case that the signal and the noise have very different characteristics and, in this case, we can try to reduce the amount of noise via filtering. Typically, if we look in the time domain:
 * the clean signal is varying slowly and smoothly
 * the noise is low-amplitude with respect to the signal and it varies very fast from one sample to the next.

## A generator for smooth noisy signals

The following function can be used to generate an $N$-point smooth signal together with a noise-corrupted version at the specified signal to noise ratio; the spectrum of the smooth signal will contain most of its energy in the $[-B\pi, B\pi]$ range. You don't need to worry about how the function works, simply use it as a black box.

In [ ]:
def sig_gen(N: int, SNR: float, B=0.04, x=None) -> [np.ndarray, np.ndarray]:
    if x is None:
        X = np.r_[0, np.random.uniform(-1, 1, 2 * int(N * B) + 1)]
        x = np.real(np.fft.ifft(X, 2*N))[:N] / np.sqrt(2 * B / 3 / N)
    a = np.sqrt((3.0 / 8.0) / np.power(10, SNR / 10)) 
    return x, x + np.random.uniform(-a, a, len(x))

In [ ]:
# Only for the very curious students!!

def sig_gen(N: int, SNR: float, B=0.04, x=None) -> [np.ndarray, np.ndarray]:
    if x is None:
        # build a smooth signal by creating a DFT vector X of length 2 * int(N * B) + 2 and
        #  taking a 2N-point inverse DFT. The resulting signal will be bandlimited to 2pi * B.
        # X[0] = 0 to ensure zero mean; other values randomly distributed over [-1, 1]
        X = np.r_[0, np.random.uniform(-1, 1, 2 * int(N * B) + 1)]
        # at this point the signal's energy is the number of nonzero DFT coefficients times
        #  their variance, divided by the number of time samples: (2NB)(1/3)/(2N) = B / 3
        # We take the real part only, so energy is B / 6
        # To get a signal with approx unit peak, normalize by sqrt of twice the power
        #  (pretending the signal is sinusoidal so that peak = RMS * sqrt(2))
        x = np.real(np.fft.ifft(X, 2*N))[:N] / np.sqrt(2 * B / 3 / N)
    # amplitude of the noise from the desired SNR, knowing that now signal energy is 0.125
    a = np.sqrt((3.0 / 8.0) / np.power(10, SNR / 10)) 
    return x, x + np.random.uniform(-a, a, len(x))

Use the following interactive widget to play with the SNR and the B parameters and try to get a feel for their effect on the  signal generated by the function:

In [ ]:
def display(SNR=15, B=0.02):
    x, x_hat = sig_gen(1000, SNR, B, display.prev[1] if B == display.prev[0] else None)
    display.prev = [B, x]
    plt.plot(x, 'C0', lw=2, label='clean');
    plt.plot(x_hat, 'C3', lw=1, label='noisy');
    plt.ylim(-1.2,1.2);
    plt.legend(loc="upper right");

display.prev = [0, None]
    
interact(display, SNR=(0.0, 50.0), B=(0.01, 0.09, 0.01));

### Exercise: checking the SNR

Given a noise-corrupted signal $\hat{x}[n] = x[n] + \eta[n]$, the signal-to-noise ratio is expressed in dB and is computed as 

$$
    \text{SNR}_{\hat{x}} = 10 \log_{10}\left(\frac{E_x}{E_\eta}\right)
$$

where $E_x$ is the energy of the clean signal and $E_\eta$ is the energy of the noise. 

Generate a noisy signal and verify numerically that the SNR of the sequence returned by `sig_gen()` is indeed close to the SNR passed as an argument to the function.

In [ ]:
N, SNR = 1000, 30
x, x_hat = sig_gen(N, SNR)
E_x = ...
E_eta = ...
SNR_exp = ...

## Denoising via averaging

As a first idea, try to remove the noise by replacing each sample with a local average of the data

In [ ]:
#...

## Denoising with a recursive filter

Denoising is a lowpass operation, in the sense that it removes the fast variations due to the noise while preserving the smooth, slowly-varying signal envelope. Since an RC circuit is a lowpass filter, use a discretized recursive filter to remove the noise